In [2]:
import pandas as pd
import time
from tqdm import tqdm
from collections import defaultdict
import matplotlib.pyplot as plt
from IPython.display import clear_output, display
from ipywidgets import IntProgress, IntText, Text

data_path = 'HGDP/hgdp/HGDP_FinalReport_Forward.txt'
pops_path = 'HGDP/hgdp/HGDP-CEPH-ID_populations.csv'
n, m = 1043, 660918

pops = pd.read_csv(pops_path)

# Read and pre-process data SNP by SNP

In [2]:
dsizebox = IntText(value=1000, description='dump_size:')
display(dsizebox)

IntText(value=1000, description='dump_size:')

# Merge

# Combine w/ population/geographic data

In [10]:
column_names = pd.read_csv('full_cleaned_data.csv', nrows=0)
print(column_names)

# for i, chunk in enumerate(reader):
#     print(chunk.columns)
#     input()

KeyboardInterrupt: 